In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print(pd.__version__)

2.2.1


In [5]:
files = glob.glob('../data/optuna_results*.txt')
columns = 'Study,model name,metric to optimize,rank,embed. size,learning rate,optimizer,batch size,return_L2,coef vectors,coef biases,"part id",trial number,epoch number,train loss,train auc,validation loss,validation auc,test loss,test auc,epoch train time,validation time,criterion,dataset'.split(',')

In [ ]:
dfs = []
for file in files:
  print(f'Loading {file}')
  dfs.append(pd.read_csv(file, names=columns))
df = pd.concat(dfs, axis=0, ignore_index=True)
df.head()

In [ ]:
datasets = df['dataset'].unique()
datasets

In [ ]:
experiment_cols = ['dataset', 'metric to optimize', 'model name', 'rank', 'embed. size']

def best_config_test_metric(group):
  if group['metric to optimize'].iloc[0] == 'auc':
    costs = group[['validation auc', 'test auc']]
    agg = pd.Series.idxmax
  else:
    costs = group[['validation loss', 'test loss']]
    agg = pd.Series.idxmin
  costs.columns = ['validation', 'test']

  idx_opt = agg(costs['validation'])
  opt = costs.loc[idx_opt, 'test']
  return pd.Series([opt], name='test loss').to_frame()

test_loss_df = df \
  .groupby(experiment_cols).apply(best_config_test_metric) \
  .reset_index() \
  .set_index('dataset')